# Process Handbook

This notebook processes the example handbook (CAHRC_HR_Manual.txt).  This is done in a simple fashion using the following heuristic: If a line of text consisting of less than 5 words is followed by paragraphs of text the assume the line of text with less than 5 words is a topic (i.e. the topic of a question an employee might ask) and that the paragraphs of text are the answer to that question (called action_text for the lack of a better term).

When a topic and action_text are found these are stored in Cloud Datastore as a key-value pair with the topic as the key and the action_text as the value.

In [1]:
!pip uninstall -y google-cloud-datastore

Skipping google-cloud-datastore as it is not installed.


In [2]:
!pip install google-cloud-datastore

    100% |████████████████████████████████| 143kB 11.1MB/s ta 0:00:01
    100% |████████████████████████████████| 143kB 16.1MB/s ta 0:00:01
pandas-gbq 0.3.0 has requirement google-cloud-bigquery>=0.28.0, but you'll have google-cloud-bigquery 0.23.0 which is incompatible.
google-cloud-bigquery 0.23.0 has requirement google-cloud-core<0.24dev,>=0.23.1, but you'll have google-cloud-core 1.6.0 which is incompatible.
tensorboard 1.8.0 has requirement bleach==1.5.0, but you'll have bleach 2.1.2 which is incompatible.
tensorboard 1.8.0 has requirement html5lib==0.9999999, but you'll have html5lib 1.1 which is incompatible.
  Found existing installation: six 1.10.0
    Uninstalling six-1.10.0:
      Successfully uninstalled six-1.10.0
  Found existing installation: google-auth 1.21.1
    Uninstalling google-auth-1.21.1:
      Successfully uninstalled google-auth-1.21.1
  Found existing installation: google-cloud-core 0.28.1
    Uninstalling google-cloud-core-0.28.1:
      Successfully uninstal

Hit Reset Session > Restart, then resume with the following cells. 

In [7]:
from google.cloud import datastore
datastore_client = datastore.Client()
import csv

In [44]:
with open('netflix_titles.csv') as csvfile:
  csvreader = csv.reader(csvfile, delimiter=',', quotechar='"')
  line_count = 0
  header=''
  for row in csvreader:
    if line_count == 0:
      header = row
      print header
      line_count += 1
      continue
            
    kind = 'netflix'
    topic_key = datastore_client.key(kind, row[0])
    topic = datastore.Entity(key=topic_key)
    for i in range(len(header)):
      topic[header[i]] = row[i]
      
    print topic
    datastore_client.put(topic)
    print('Saved {}: {}'.format(topic.key.name, topic))
    line_count += 1
    
    #for dev and testing
    if line_count == 4:
      break
      
  #for validation
  print "\n"
  query = datastore_client.query(kind='netflix')
  results = list(query.fetch())
  print results

['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added', 'release_year', 'rating', 'duration', 'listed_in', 'description']
<Entity('netflix', 's1') {'rating': 'TV-MA', 'show_id': 's1', 'release_year': '2020', 'title': '3%', 'country': 'Brazil', 'director': '', 'cast': 'Jo\xc3\xa3o Miguel, Bianca Comparato, Michel Gomes, Rodolfo Valente, Vaneza Oliveira, Rafael Lozano, Viviane Porto, Mel Fronckowiak, Sergio Mamberti, Zez\xc3\xa9 Motta, Celso Frateschi', 'listed_in': 'International TV Shows, TV Dramas, TV Sci-Fi & Fantasy', 'duration': '4 Seasons', 'date_added': 'August 14, 2020', 'type': 'TV Show', 'description': 'In a future where the elite inhabit an island paradise far from the crowded slums, you get one chance to join the 3% saved from squalor.'}>
Saved s1: <Entity('netflix', 's1') {'rating': 'TV-MA', 'show_id': 's1', 'release_year': '2020', 'title': '3%', 'country': 'Brazil', 'director': '', 'cast': 'Jo\xc3\xa3o Miguel, Bianca Comparato, Michel Gomes, Rodolfo Valen